In [1]:
import json

In [2]:
from pyspark.sql import SparkSession

In [3]:
config = json.load(open("config.json"))

In [4]:
spark = SparkSession.builder\
    .config("spark.jars", config['JDBC_PATH']) \
    .config("spark.executor.extraClassPath", config['JDBC_PATH'])\
    .getOrCreate()

24/01/09 01:10:58 WARN Utils: Your hostname, przemyslaw-Latitude-E7450 resolves to a loopback address: 127.0.1.1; using 192.168.1.110 instead (on interface wlp2s0)
24/01/09 01:10:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/09 01:10:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 01:11:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/09 01:11:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/01/09 01:11:01 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/01/09 01:11:01 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
dbtable = "service"

In [6]:
df_service = spark.read.format("jdbc")\
        .option("url",config['SRC_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['SRC_DB_USER'])\
        .option("password", config['SRC_DB_PASSWORD'])\
        .load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [7]:
df_service.createOrReplaceTempView(dbtable)

In [8]:
df_service.show()

+----------+------------+------------+------------+------------+
|service_id|inventory_id|service_type|service_date|service_cost|
+----------+------------+------------+------------+------------+
|         1|           1| tire change|  2016-03-01|    157.5000|
|         2|           1| oil service|  2016-03-01|    105.0000|
|         3|           2| tire change|  2016-03-01|    157.5000|
|         4|           2| oil service|  2016-03-01|    105.0000|
|         5|           3| tire change|  2016-03-01|    157.5000|
|         6|           3| oil service|  2016-03-01|    157.5000|
|         7|           4| tire change|  2016-03-01|    157.5000|
|         8|           4| oil service|  2016-03-01|    105.0000|
|         9|           5| tire change|  2016-03-01|    157.5000|
|        10|           5| oil service|  2016-03-01|    210.0000|
|        11|           6| tire change|  2016-03-01|    157.5000|
|        12|           6| oil service|  2016-03-01|    157.5000|
|        13|           8|

In [10]:
dbtable = "dim_inventory"

In [11]:
df_dim_inventory = spark.read.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .load()

In [12]:
df_dim_inventory.createOrReplaceTempView(dbtable)

In [13]:
dbtable = "dim_calendar"

In [14]:
df_dim_calendar = spark.read.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", dbtable)\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .load()

In [15]:
df_dim_calendar.createOrReplaceTempView(dbtable)

In [16]:
fact_service = spark.sql("""
SELECT
ROW_NUMBER() OVER(ORDER BY s.service_id) AS service_id,
i.inventory_id AS service_inventory_id,
c.calendar_id AS service_date_id,
s.service_id AS service_key,
s.service_type AS service_type,
s.service_cost AS service_cost
FROM service AS s
JOIN dim_inventory AS i ON s.inventory_id = i.inventory_key
JOIN dim_calendar AS c ON s.service_date = c.calendar_full_date
""")

In [17]:
fact_service.show()

24/01/09 01:12:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:12:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:12:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:12:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:12:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:12:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0

+----------+--------------------+---------------+-----------+------------+------------+
|service_id|service_inventory_id|service_date_id|service_key|service_type|service_cost|
+----------+--------------------+---------------+-----------+------------+------------+
|         1|                   1|             61|          1| tire change|    157.5000|
|         2|                   1|             61|          2| oil service|    105.0000|
|         3|                   2|             61|          3| tire change|    157.5000|
|         4|                   2|             61|          4| oil service|    105.0000|
|         5|                   3|             61|          5| tire change|    157.5000|
|         6|                   3|             61|          6| oil service|    157.5000|
|         7|                   4|             61|          7| tire change|    157.5000|
|         8|                   4|             61|          8| oil service|    105.0000|
|         9|                   5

In [18]:
fact_service.createOrReplaceTempView("fact_service")

In [19]:
spark.sql("""
ALTER TABLE fact_service ADD PRIMARY KEY (service_id)
ALTER TABLE dim_inventory ADD PRIMARY KEY (inventory_id)
ALTER TABLE dim_calendar ADD PRIMARY KEY (calendar_id)

ALTER TABLE fact_service
ADD CONSTRAINT fact_service_FK
FOREIGN KEY (service_inventory_id) REFERENCES dim_inventory (inventory_id)
          
ALTER TABLE fact_service
ADD CONSTRAINT fact_service_FK2
FOREIGN KEY (service_date_id) REFERENCES dim_calendar (calendar_id) 
""")

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'PRIMARY'.(line 2, pos 29)

== SQL ==

ALTER TABLE fact_service ADD PRIMARY KEY (service_id)
-----------------------------^^^
ALTER TABLE dim_inventory ADD PRIMARY KEY (inventory_id)

ALTER TABLE fact_service
ADD CONSTRAINT fact_service_FK
FOREIGN KEY (service_inventory_id) REFERENCES dim_inventory (inventory_id) 


In [19]:
fact_service.write.format("jdbc")\
        .option("url",config['DST_DB_URL'])\
        .option("driver", "com.mysql.jdbc.Driver")\
        .option("dbtable", "fact_service")\
        .option("user", config['DST_DB_USER'])\
        .option("password", config['DST_DB_PASSWORD'])\
        .save()

24/01/09 01:13:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:13:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:13:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:13:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:13:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 01:13:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 0